In [26]:
!pip install minio boto3 pandas pyarrow -q

print("Либы встали успешно!")

Либы встали успешно!


In [39]:
# Это для правильного отображения датафрейма
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

In [16]:
# endpoint для подключения к MinIO
# разные варианты подключения

def get_minio_endpoint():
    """Определяет правильный эндпоинт для подключения к MinIO из контейнера"""
    
    endpoints_to_try = [
        "172.17.0.1:9000",            # Linux Docker bridge - рабоать будет через это соединение
        "minio:9000",                  # Если контейнеры в одной сети
        "localhost:9000",              # Если запущено не в контейнере
    ]
    
    for endpoint in endpoints_to_try:
        try:
            host = endpoint.split(":")[0]
            socket.setdefaulttimeout(2)
            socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect((host, 9000))
            print(f"✓ Успешное подключение к {endpoint}")
            return endpoint
        except (socket.error, socket.timeout) as e:
            print(f"✗ Не удалось подключиться к {endpoint}: {e}")
            continue
    
    # получить IP хоста
    try:
        import subprocess
        result = subprocess.run(['ip', 'route', 'show', 'default'], 
                              capture_output=True, text=True)
        gateway_ip = result.stdout.split()[2]
        print(f"Используем gateway IP: {gateway_ip}")
        return f"{gateway_ip}:9000"
    except:
        return "host.docker.internal:9000"

# получить endpoint
MINIO_ENDPOINT = get_minio_endpoint()
print(f"\nИспользуемый endpoint: {MINIO_ENDPOINT}")

✓ Успешное подключение к 172.17.0.1:9000

Используемый endpoint: 172.17.0.1:9000


In [17]:
# Учетные данные MinIO
MINIO_ACCESS_KEY = "****USER****"
MINIO_SECRET_KEY = "****PASSWORD****"

# клиент MinIO
minio_client = Minio(
    endpoint=MINIO_ENDPOINT,
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False  # Без SSL для локального использования
)

# проверка подключения, получая список бакетов
try:
    buckets = minio_client.list_buckets()
    print("✓ Подключение к MinIO успешно!")
    print(f"Существующие бакеты: {[b.name for b in buckets]}")
except S3Error as e:
    print(f"✗ Ошибка подключения к MinIO: {e}")
except Exception as e:
    print(f"✗ Общая ошибка: {e}")

✓ Подключение к MinIO успешно!
Существующие бакеты: ['hw-kotsyuba']


In [23]:
# Датафрейм с инфой о студентах нашего курса (фейковые имена, если что)
data = {
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'student_name': [
        'Иванов Иван', 'Петрова Мария', 'Сидоров Алексей', 
        'Козлова Елена', 'Новиков Дмитрий', 'Морозова Анна',
        'Волков Сергей', 'Лебедева Ольга', 'Соколов Андрей', 'Павлова Наталья'
    ],
    'course': ['Data Engineering'] * 10,
    'homework_score': [95, 88, 92, 78, 85, 90, 82, 96, 88, 91],
    'exam_score': [90, 85, 88, 75, 80, 92, 78, 94, 86, 89],
    'enrollment_date': pd.date_range(start='2024-01-15', periods=10, freq='D'),
    'is_active': [True, True, True, False, True, True, True, True, False, True]
}

df = pd.DataFrame(data)

# Добавляем вычисляемый столбец
df['total_score'] = (df['homework_score'] * 0.4 + df['exam_score'] * 0.6).round(2)
df['grade'] = pd.cut(df['total_score'], 
                     bins=[0, 60, 70, 80, 90, 100], 
                     labels=['F', 'D', 'C', 'B', 'A'])


print(df.dtypes)
print(f"\n{'-'*60}")

print(df.to_string(index=False))
print(f"\n{'-'*60}")

id                          int64
student_name               object
course                     object
homework_score              int64
exam_score                  int64
enrollment_date    datetime64[ns]
is_active                    bool
total_score               float64
grade                    category
dtype: object

------------------------------------------------------------
 id    student_name           course  homework_score  exam_score enrollment_date  is_active  total_score grade
  1     Иванов Иван Data Engineering              95          90      2024-01-15       True         92.0     A
  2   Петрова Мария Data Engineering              88          85      2024-01-16       True         86.2     B
  3 Сидоров Алексей Data Engineering              92          88      2024-01-17       True         89.6     B
  4   Козлова Елена Data Engineering              78          75      2024-01-18      False         76.2     C
  5 Новиков Дмитрий Data Engineering              85          8

### **Сохранить в Minio**

#### **CSV**

In [46]:
def save_df_to_minio(client, bucket_name, object_name, dataframe):
    """Сохраняет df в MinIO в формате CSV или Parquet"""
    ext = object_name.lower().split('.')[-1]
    
    if ext not in ('csv', 'parquet'):
        print(f"✗ Неподдерживаемый формат: .{ext}")
        return False
    
    try:
        buffer = BytesIO()
        if ext == 'csv':
            dataframe.to_csv(buffer, index=False, encoding='utf-8')
            content_type = 'text/csv'
        else:
            dataframe.to_parquet(buffer, index=False)
            content_type = 'application/octet-stream'
        
        data = buffer.getvalue()
        client.put_object(bucket_name, object_name, BytesIO(data), len(data), content_type)
        
        print(f"✓ Файл '{object_name}' сохранён | {len(data)} байт")
        return True
    except Exception as e:
        print(f"✗ Ошибка: {e}")
        return False

# Сохранение df как csv и паркет
save_df_to_minio(minio_client, BUCKET_NAME, "data.csv", df)
save_df_to_minio(minio_client, BUCKET_NAME, "data.parquet", df)

✓ Файл 'data.csv' сохранён | 848 байт
✓ Файл 'data.parquet' сохранён | 6883 байт


True

### Функция для вывода файлов в MinIO

In [45]:
def list_bucket_objects(client, bucket_name):
    """Выводит список объектов в бакете"""
    print(f"\n{'='*60}")
    print(f"Содержимое бакета '{bucket_name}':")
    print(f"{'='*60}")
    
    try:
        objects = client.list_objects(bucket_name, recursive=True)
        total_size = 0
        object_count = 0
        
        print(f"{'Имя файла':<40} {'Размер':<15} {'Дата изменения'}")
        print(f"{'-'*40} {'-'*15} {'-'*25}")
        
        for obj in objects:
            size_str = f"{obj.size:,} байт"
            date_str = obj.last_modified.strftime("%Y-%m-%d %H:%M:%S") if obj.last_modified else "N/A"
            print(f"{obj.object_name:<40} {size_str:<15} {date_str}")
            total_size += obj.size
            object_count += 1
        
        print(f"{'-'*60}")
        print(f"Всего объектов: {object_count}, Общий размер: {total_size:,} байт")
        
    except S3Error as e:
        print(f"✗ Ошибка при получении списка объектов: {e}")

# Выводим список файлов в бакете
list_bucket_objects(minio_client, BUCKET_NAME)


Содержимое бакета 'hw-kotsyuba':
Имя файла                                Размер          Дата изменения
---------------------------------------- --------------- -------------------------
students_data.csv                        848 байт        2025-12-20 19:56:26
students_data.parquet                    7,123 байт      2025-12-20 19:56:28
------------------------------------------------------------
Всего объектов: 2, Общий размер: 7,971 байт


### Загрузить файл из MinIO

In [43]:
def read_file_from_minio(client, bucket_name, object_name):
    """Читает нужный файл из MinIO и возвращает df"""
    ext = object_name.lower().split('.')[-1]
    readers = {'csv': pd.read_csv, 'parquet': pd.read_parquet}
    
    if ext not in readers:
        print(f"✗ Неподдерживаемый формат: .{ext}")
        return None
    
    try:
        response = client.get_object(bucket_name, object_name)
        df = readers[ext](BytesIO(response.read()))
        response.close()
        response.release_conn()
        
        print(f"✓ Файл '{object_name}' прочитан | {df.shape[0]} строк, {df.shape[1]} столбцов")
        return df
    except Exception as e:
        print(f"✗ Ошибка: {e}")
        return None

In [42]:
df = read_file_from_minio(minio_client, BUCKET_NAME, "students_data.csv")
display(df.head())

✓ Файл 'students_data.csv' прочитан | 10 строк, 9 столбцов


,id,student_name,course,homework_score,exam_score,enrollment_date,is_active,total_score,grade
0,1,Иванов Иван,Data Engineering,95,90,2024-01-15,True,92.0,A
1,2,Петрова Мария,Data Engineering,88,85,2024-01-16,True,86.2,B
2,3,Сидоров Алексей,Data Engineering,92,88,2024-01-17,True,89.6,B
3,4,Козлова Елена,Data Engineering,78,75,2024-01-18,False,76.2,C
4,5,Новиков Дмитрий,Data Engineering,85,80,2024-01-19,True,82.0,B
